In [28]:
from typing import Iterable, Literal

import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.figure import Figure, SubFigure
from matplotlib.ticker import LogFormatter

from common import data_folder, figure_folder, item_names_T1, item_names_T2, item_names_T3

plt.rcParams.update({
    "font.sans-serif": "Source Han Sans CN",
    "figure.dpi": 144,
    "figure.constrained_layout.use": True,
    "savefig.bbox": "tight",
})


class LogFormatterPlain(LogFormatter):
    def __init__(self, format_str_or_function="{x:,.0f}", *args, **kwargs):
        super().__init__(*args, **kwargs)
        if isinstance(format_str_or_function, str):
            self.function = lambda x: format_str_or_function.format(x=x)
        else:
            self.function = format_str_or_function

    def _num_to_string(self, x, vmin, vmax):
        return self.function(x)


In [29]:
df_T3 = pd.read_csv(data_folder / "掉单一蓝材料的活动.csv", index_col=0, parse_dates=["作战开放时间"])
df_T2 = pd.read_csv(data_folder / "掉两种绿材料的活动.csv", index_col=0, parse_dates=["作战开放时间"])
df_T1 = pd.read_csv(data_folder / "掉全部白材料的活动.csv", index_col=0, parse_dates=["作战开放时间"])


In [30]:
# 蓝材料图表

def plot(fig: Figure | SubFigure, item_name: str, group: pd.DataFrame):
    ax1 = fig.subplots()
    ax2 = ax1.twinx()

    left_color = "tab:blue"
    right_color = "tab:red"

    ax1.set_title(f"{item_name}")
    ax1.set_xticks(range(len(group)))
    ax1.set_xticklabels([f"{zone_name} {stage_name}" for zone_name, stage_name in group[["活动名称", "作战名称"]].to_records(index=False)],
                        rotation=45, horizontalalignment="right")
    ax1.set_xlim(-1, len(group) - 1 + 1)
    ax1.set_ylabel("样本数", color=left_color)
    ax1.tick_params(axis="y", which="both", colors=left_color)
    ax1.set_yscale("log")
    ax1.yaxis.set_major_formatter(LogFormatterPlain())
    ax1.yaxis.set_minor_formatter(LogFormatterPlain())

    ax2.set_ylabel("单位理智掉落物品数量\n单件期望理智", color=right_color)
    ax2.tick_params(axis="y", which="both", colors=right_color)
    ax2.yaxis.set_major_formatter(lambda x, _: f"{x:.5f}\n{1/x:.4f}")

    ax1.bar(range(len(group)), group["样本数"], color=left_color, alpha=0.7, label="样本数")
    ax2.plot(range(len(group)), group["单位理智掉落物品数量"], color=right_color, label="单位理智掉落物品数量\n单件期望理智", marker="o")
    for i, x in enumerate(group["单位理智掉落物品数量"]):
        if i % 2 == 0:
            verticalalignment = "top"
            xytext = (0, -8)
        else:
            verticalalignment = "bottom"
            xytext = (0, 8)
        ax2.annotate(f"{x:.5f}\n{1/x:.4f}", (i, x),
                     horizontalalignment="center", verticalalignment=verticalalignment, fontsize=9, textcoords="offset points", xytext=xytext,
                     bbox=dict(boxstyle="round,pad=0.2", edgecolor="none", facecolor="white", alpha=0.7))
    fig.legend(loc="lower center", bbox_to_anchor=(1, 1), bbox_transform=ax1.transAxes)


grouped = sorted(df_T3.groupby("作战掉落物品名称"), key=lambda x: item_names_T3.index(x[0]))  # type: ignore

# 大图
fig = plt.figure(figsize=(12, 18 * 7))
subfigs: Iterable[SubFigure] = fig.subfigures(18, 1)  # type: ignore
fig.suptitle("SideStory历史掉率（掉单一蓝材料的作战）", verticalalignment="bottom")
for subfig, (item_name, group) in zip(subfigs, grouped):
    plot(subfig, item_name, group)
fig.savefig(figure_folder / "蓝材料掉落图表.png")
plt.close(fig)

# 单图
for item_name, group in grouped:
    fig = plt.figure(figsize=(12, 7))
    plot(fig, item_name, group)
    fig.savefig(figure_folder / f"蓝材料图表/{item_name}.png")
    plt.close(fig)


In [31]:
# 绿材料图表

def plot(fig: Figure | SubFigure, item_name: str, group: pd.DataFrame, drop_type: Literal["主", "副"]):
    ax1 = fig.add_subplot()
    ax2 = ax1.twinx()

    left_color = "tab:blue"
    right_color = "tab:red"

    ax1.set_title(f"{item_name}（作为{drop_type}掉落）")
    ax1.set_xticks(range(len(group)))
    ax1.set_xticklabels([f"{zone_name} {stage_name}" for zone_name, stage_name in group[["活动名称", "作战名称"]].to_records(index=False)],
                        rotation=45, horizontalalignment="right")
    ax1.set_xlim(-1, len(group) - 1 + 1)
    ax1.set_ylabel("样本数", color=left_color)
    ax1.tick_params(axis="y", which="both", colors=left_color)
    ax1.set_yscale("log")
    ax1.yaxis.set_major_formatter(LogFormatterPlain())
    ax1.yaxis.set_minor_formatter(LogFormatterPlain())

    ax2.set_ylabel(f"单位理智{drop_type}掉落数量\n单件期望理智", color=right_color)
    ax2.tick_params(axis="y", which="both", colors=right_color)
    ax2.yaxis.set_major_formatter(lambda x, _: f"{x:.5f}\n{1/x:.4f}")

    ax1.bar(range(len(group)), group["样本数"], color=left_color, alpha=0.7, label="样本数")
    ax2.plot(range(len(group)), group[f"单位理智{drop_type}掉落数量"], color=right_color, label=f"单位理智{drop_type}掉落数量\n单件期望理智", marker="o")
    for i, x in enumerate(group[f"单位理智{drop_type}掉落数量"]):
        if i % 2 == 0:
            verticalalignment = "top"
            xytext = (0, -8)
        else:
            verticalalignment = "bottom"
            xytext = (0, 8)
        ax2.annotate(f"{x:.5f}\n{1/x:.4f}", (i, x),
                     horizontalalignment="center", verticalalignment=verticalalignment, fontsize=7, textcoords="offset points", xytext=xytext,
                     bbox=dict(boxstyle="round,pad=0.2", edgecolor="none", facecolor="white", alpha=0.5))
    fig.legend(loc="lower center", bbox_to_anchor=(1, 1), bbox_transform=ax1.transAxes)


grouped = [
    *(("主", item_name, group) for item_name, group in sorted(df_T2.groupby("主掉落物品名称"), key=lambda x: item_names_T2.index(x[0]))),  # type: ignore
    *(("副", item_name, group) for item_name, group in sorted(df_T2.groupby("副掉落物品名称"), key=lambda x: item_names_T2.index(x[0]))),  # type: ignore
]

# 大图
fig = plt.figure(figsize=(2 * 15, 6 * 7))
subfigs: Iterable[SubFigure] = fig.subfigures(6, 2).transpose().reshape(-1)
fig.suptitle("SideStory历史掉率（掉两种绿材料的作战）", y=1.02)
for subfig, (drop_type, item_name, group) in zip(subfigs, grouped):
    plot(subfig, item_name, group, drop_type)
fig.savefig(figure_folder / "绿材料掉落图表.png")
plt.close(fig)

# 单图
for drop_type, item_name, group in grouped:
    fig = plt.figure(figsize=(15, 7))
    plot(fig, item_name, group, drop_type)
    fig.savefig(figure_folder / f"绿材料{drop_type}掉落图表/{item_name}.png")
    plt.close(fig)


In [32]:
fig = plt.figure(figsize=(40, 7))
ax1 = fig.add_subplot()
ax2 = ax1.twinx()

left_color = "black"

ax1.set_title(f"SideStory历史掉率（掉全部白材料的作战）")
ax1.set_xticks(range(len(df_T1)))
ax1.set_xticklabels([f"{zone_name} {stage_name}" for zone_name, stage_name in df_T1[["活动名称", "作战名称"]].to_records(index=False)],
                    rotation=45, horizontalalignment="right")
ax1.set_xlim(-1, len(df_T1) - 1 + 1)
ax1.set_ylabel("样本数", color=left_color)
ax1.tick_params(axis="y", which="both", colors=left_color)
ax1.set_yscale("log")
ax1.yaxis.set_major_formatter(LogFormatterPlain())
ax1.yaxis.set_minor_formatter(LogFormatterPlain())

ax2.set_ylabel("单位理智掉落物品数量\n单件期望理智")
ax2.yaxis.set_major_formatter(lambda x, _: f"{x:.2f}\n{1/x:.2f}" if x != 0 else f"0.00\n+∞")
ax2.set_ylim(0, 0.08)
ax2.yaxis.minorticks_on()
ax2.grid(axis="y", which="major")
ax2.grid(axis="y", which="minor", linewidth=0.5)

bar = ax1.bar(range(len(df_T1)), df_T1["样本数"], color=left_color, alpha=0.3, label="样本数")
for i, item_name in enumerate(item_names_T1):
    markers = ["o", "s", "D", "^", "v", "P"]
    ax2.plot(range(len(df_T1)), 1 / df_T1[f"{item_name}单件期望理智"], label=f"{item_name}单位理智掉落物品数量\n{item_name}单件期望理智", marker=markers[i])
fig.legend(loc="lower right", ncols=7, bbox_to_anchor=(1, 1), bbox_transform=ax1.transAxes)

fig.savefig(figure_folder / "白材料掉落图表.png")
plt.close(fig)
